In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import json

# Constants
MODEL_NAME = "gpt2"
BATCH_SIZE = 1
EPOCHS = 1
PROMPT_TOKEN = "[QUESTIONANSWERING]"
MAX_LEN = 512

# Soft Prompt Vocabulary
soft_prompt_vocab = ["[QUESTIONANSWERING]"]  

# Create a word2idx dictionary for the soft prompt vocabulary
soft_prompt_word2idx = {word: idx for idx, word in enumerate(soft_prompt_vocab)}

num_prompts = len([soft_prompt_word2idx[word] for word in PROMPT_TOKEN.split()])
prompt_id = torch.tensor([soft_prompt_word2idx[word] for word in PROMPT_TOKEN.split()])

# Model Architecture
class GPT2WithSoftPrompt(torch.nn.Module):
    def __init__(self, model_name, num_prompts, embedding_size=768):
        super().__init__()
        self.gpt2 = GPT2LMHeadModel.from_pretrained(model_name)
        self.soft_prompt = torch.nn.Embedding(num_prompts, embedding_size)

    def forward(self, input_ids, prompt_ids):
        prompt_embeddings = self.soft_prompt(prompt_ids)
        base_embeddings = self.gpt2.transformer.wte(input_ids)
        embeddings = torch.cat([prompt_embeddings, base_embeddings.squeeze(0)], dim=0)
        outputs = self.gpt2(inputs_embeds=embeddings)
        return outputs

    
# Load data from a JSON file
def load_data_from_json(json_file):
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)
    
    context_list = [dp["context"] for item in data["data"] for dp in item["paragraphs"]]
    question_list = [dp["qas"][0]["question"] for item in data["data"] for dp in item["paragraphs"]]
    answer_list = [dp["qas"][0]["answers"][0]["text"] if dp["qas"] and dp["qas"][0]["answers"] else "" for item in data["data"] for dp in item["paragraphs"]]

    return context_list[:500], question_list[:500], answer_list[:500]

# Data Loading and Preprocessing
def load_and_preprocess_data(json_file, num_prompts):
    context_list, question_list, answer_list = load_data_from_json(json_file)

    # Perform preprocessing on the data
    tokenized_question = []
    tokenized_answer = []

    tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

    for context, question, answer in zip(context_list, question_list, answer_list):
        # Tokenize context, question, and answer using the GPT-2 tokenizer
        context_tokens = tokenizer.encode(context, truncation=True, max_length=MAX_LEN)
        question_tokens = tokenizer.encode(question, truncation=True, max_length=MAX_LEN)
        answer_tokens = tokenizer.encode(answer, truncation=True, max_length=MAX_LEN)

        padded_article = question_tokens + [tokenizer.eos_token_id] * (MAX_LEN-1 - len(question_tokens))
        padded_summary = answer_tokens + [tokenizer.eos_token_id] * (MAX_LEN - len(answer_tokens))

        tokenized_question.append(padded_article)
        tokenized_answer.append(padded_summary)

    return tokenized_question, tokenized_answer


# Load and preprocess the data
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
tokenized_articles_train,tokenized_summaries_train = load_and_preprocess_data("train-v2.0.json",num_prompts)
tokenized_articles_validation,tokenized_summaries_validation = load_and_preprocess_data("dev-v2.0.json", num_prompts)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# # Model Initialization
model = GPT2WithSoftPrompt(MODEL_NAME, num_prompts).to(device)


In [2]:
from tqdm import tqdm

# Hyperparameters
BATCH_SIZE = 1
EPOCHS = 10
GRADIENT_ACCUMULATION_STEPS = 1
GRADIENT_CLIP_NORM = 1.0
EARLY_STOPPING_PATIENCE = 2
prompt_id = prompt_id.to(device)

from torch.nn import CrossEntropyLoss

def fine_tune_on_summarization(model, train_articles, train_summaries, val_articles, val_summaries):
    optimizer = torch.optim.Adam(model.soft_prompt.parameters())

    best_val_loss = float('inf')
    no_improvement_epochs = 0

    for epoch in range(EPOCHS):
        model.train()

        # Gradient accumulation initialization
        optimizer.zero_grad()
        accumulated_loss = 0
        loss = 0
        # Use tqdm for progress bar
        with tqdm(enumerate(zip(train_articles, train_summaries)), total=len(train_articles), desc=f"Epoch {epoch + 1}/{EPOCHS}", unit="batch") as progress:
            train_percentage_matched = 0
            train_percentage_matched_ct = 0
            for idx, (article, summary) in progress:
                input_ids = torch.tensor(article).to(device)
                labels = torch.tensor(summary).to(device)
                outputs = model(input_ids, prompt_id)

                ignore_index = tokenizer.eos_token_id
                loss += CrossEntropyLoss(ignore_index=ignore_index)(outputs.logits, labels)

                # Metrics
                set1 = set(torch.argmax(outputs.logits, dim=1).cpu().numpy())
                set2 = set(labels.cpu().numpy())

                # Calculate the intersection of sets
                intersection = set1.intersection(set2)

                # Calculate the percentage of indices in the first tensor that are also in the second tensor
                percentage = (len(intersection) / len(set1)) * 100
                train_percentage_matched += percentage
                train_percentage_matched_ct += 1

                # Backpropagate losses every GRADIENT_ACCUMULATION_STEPS or at the end of the dataset
                if (idx + 1) % GRADIENT_ACCUMULATION_STEPS == 0 or idx == len(train_articles) - 1:
                    (loss / GRADIENT_ACCUMULATION_STEPS).backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_CLIP_NORM)
                    optimizer.step()
                    optimizer.zero_grad()
                    loss = 0
            
            print("Train : % Exact Match: ",train_percentage_matched/train_percentage_matched_ct)

        # Validation
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            val_percentage_matched = 0
            val_percentage_matched_ct = 0
            for article, summary in tqdm(zip(val_articles, val_summaries), total=len(val_articles), desc="Validation", unit="batch"):
                input_ids = torch.tensor(article).to(device)
                labels = torch.tensor(summary).to(device)
                outputs = model(input_ids, prompt_id)

                ignore_index = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else -100
                val_loss = CrossEntropyLoss(ignore_index=ignore_index)(outputs.logits, labels)
                total_val_loss += val_loss.item()

                # Metrics
                set1 = set(torch.argmax(outputs.logits, dim=1).cpu().numpy())
                set2 = set(labels.cpu().numpy())

                # Calculate the intersection of sets
                intersection = set1.intersection(set2)

                # Calculate the percentage of indices in the first tensor that are also in the second tensor
                percentage = (len(intersection) / len(set1)) * 100
                val_percentage_matched += percentage
                val_percentage_matched_ct += 1
        
        
        print("Val : % Exact Match: ",val_percentage_matched/val_percentage_matched_ct)
        avg_val_loss = total_val_loss / len(val_articles)
        print("Val Loss : ",avg_val_loss)

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            no_improvement_epochs = 0
        else:
            no_improvement_epochs += 1
            if no_improvement_epochs >= EARLY_STOPPING_PATIENCE:
                print(f"Early stopping after {EARLY_STOPPING_PATIENCE} epochs without improvement.")
                break

    return model

fine_tuned_model = fine_tune_on_summarization(model, tokenized_articles_train, tokenized_summaries_train, tokenized_articles_validation, tokenized_summaries_validation)



Epoch 1/10:   0%|          | 0/500 [00:00<?, ?batch/s]

Epoch 1/10: 100%|██████████| 500/500 [01:05<00:00,  7.66batch/s]


Train : % Exact Match:  2.2524384559369075


Validation: 100%|██████████| 500/500 [00:19<00:00, 25.48batch/s]


Val : % Exact Match:  2.7082617382617378
Val Loss :  11.22950127029419


...

Val : % Exact Match:  2.622873874491521
Val Loss :  9.587493703842163


Epoch 10/10: 100%|██████████| 500/500 [01:01<00:00,  8.11batch/s]


Train : % Exact Match:  2.401964032699326


Validation: 100%|██████████| 500/500 [00:19<00:00, 25.60batch/s]

Val : % Exact Match:  2.7092987567987565
Val Loss :  9.532423812866211


# Saving Model

In [3]:
# Save the fine-tuned model
torch.save(fine_tuned_model.state_dict(), '2.pth')


# Loading Model

In [4]:
# Initialize a new instance of the model
model = GPT2WithSoftPrompt(MODEL_NAME, num_prompts).to(device)

# Load the saved model state_dict
model.load_state_dict(torch.load('2.pth'))

# Make sure the model is in evaluation mode after loading
model.eval()

GPT2WithSoftPrompt(
  (gpt2): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  )
  (soft_prompt): Embedding(1, 

# Inference

In [5]:
# Set the model to evaluation mode
model.eval()

# Input text for summarization
input_text = "Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California. Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. Her publicist, Judith Goffin, announced the news Thursday. Scroll down for video . Actress: Sally Forrest was in the 1951 Ida Lupino-directed film 'Hard, Fast and Beautiful' (left) and the 1956 Fritz Lang movie 'While the City Sleeps' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful. Some of Forrest's other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDB page. The page also indicates Forrest was in multiple Climax! and Rawhide television episodes. Forrest appeared as herself in an episode of The Ed Sullivan Show and three episodes of The Dinah Shore Chevy Show, her iMDB page says. She also starred in a Broadway production of The Seven Year Itch. City News Service reported that other stage credits included As You Like It, No, No, Nanette and Damn Yankees. Forrest married writer-producer Milo Frank in 1951. He died in 2004. She is survived by her niece, Sharon Durham, and nephews, Michael and Mark Feeney. Career: A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films ."

# Tokenize and encode the input text
input_ids = tokenizer.encode(input_text, truncation=True, max_length=1024)

# Convert the input_ids to a PyTorch tensor
input_ids = torch.tensor(input_ids)

# Generate a summary
with torch.no_grad():
    # Assuming single prompt
    outputs = model(input_ids.to(device), prompt_ids=prompt_id.to(device))
    pred_logits = outputs.logits
    print(pred_logits.shape)


# Get the token IDs with the highest probability for each position
predicted_token_ids = torch.argmax(pred_logits, dim=-1)

# Convert token IDs into words using the tokenizer
predicted_tokens = tokenizer.decode(predicted_token_ids.squeeze(0), skip_special_tokens=True)


torch.Size([360, 50257])


In [8]:
predicted_tokens

'1.. of..,... is. by woman,. the worldIs. the40s. the,s, \'. as "\'filmve film the film ofs on out the 25, the death in the Hills, California.\n\'s who name was is ",uery, was a years was been been with. She fatherist, who Lman, was that news on that\n down to the\n\n and " K, born the processsa Bin filmJones film "The Times Hard and Slow\' and19) and the film film Lang film \'The the City Sleeps\' (. Francisco Times, who was a starge of the\'sblazer anda Lupino, who was her in the roles in the such\'film darling commercial success oforious, and Again, The, and and Beautiful.\n of the\'s other films roles include\'man, The of a,, The Theite Me Love. which to the website-ovies.\n.\n film also says that\'s a the filmsax films films Thelins, series of\n\'s in a in the episode of the Big Sullivan Show, the other of The Goodosaurs and. Chase. which firstMDB profile says.\n also appeared in the number musical of the Big Dwar Itch and\n of reports reports that she actresses actors include the

# Hard Prompt

In [9]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import json

# Constants
MODEL_NAME = "gpt2"
BATCH_SIZE = 1
EPOCHS = 1
PROMPT_TOKEN = "Answer the Following Question"
MAX_LEN = 512

# Soft Prompt Vocabulary
soft_prompt_vocab = ["Answer","the","Following","Question"]  

# Create a word2idx dictionary for the soft prompt vocabulary
soft_prompt_word2idx = {word: idx for idx, word in enumerate(soft_prompt_vocab)}

num_prompts = len([soft_prompt_word2idx[word] for word in PROMPT_TOKEN.split()])
prompt_id = torch.tensor([soft_prompt_word2idx[word] for word in PROMPT_TOKEN.split()])

# Model Architecture
class GPT2WithSoftPrompt(torch.nn.Module):
    def __init__(self, model_name, num_prompts, embedding_size=768):
        super().__init__()
        self.gpt2 = GPT2LMHeadModel.from_pretrained(model_name)
        self.soft_prompt = torch.nn.Embedding(num_prompts, embedding_size)

    def forward(self, input_ids, prompt_ids):
        prompt_embeddings = self.soft_prompt(prompt_ids)
        base_embeddings = self.gpt2.transformer.wte(input_ids)
        embeddings = torch.cat([prompt_embeddings, base_embeddings.squeeze(0)], dim=0)
        outputs = self.gpt2(inputs_embeds=embeddings)
        return outputs

    
# Load data from a JSON file
def load_data_from_json(json_file):
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)
    
    context_list = [dp["context"] for item in data["data"] for dp in item["paragraphs"]]
    question_list = [dp["qas"][0]["question"] for item in data["data"] for dp in item["paragraphs"]]
    answer_list = [dp["qas"][0]["answers"][0]["text"] if dp["qas"] and dp["qas"][0]["answers"] else "" for item in data["data"] for dp in item["paragraphs"]]

    return context_list[:50], question_list[:50], answer_list[:50]

# Data Loading and Preprocessing
def load_and_preprocess_data(json_file, num_prompts):
    context_list, question_list, answer_list = load_data_from_json(json_file)

    # Perform preprocessing on the data
    tokenized_question = []
    tokenized_answer = []

    tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

    for context, question, answer in zip(context_list, question_list, answer_list):
        # Tokenize context, question, and answer using the GPT-2 tokenizer
        context_tokens = tokenizer.encode(context, truncation=True, max_length=MAX_LEN)
        question_tokens = tokenizer.encode(question, truncation=True, max_length=MAX_LEN)
        answer_tokens = tokenizer.encode(answer, truncation=True, max_length=MAX_LEN)

        padded_article = question_tokens + [tokenizer.eos_token_id] * (MAX_LEN - len(question_tokens))
        padded_summary = answer_tokens + [tokenizer.eos_token_id] * (MAX_LEN+4 - len(answer_tokens))

        tokenized_question.append(padded_article)
        tokenized_answer.append(padded_summary)

    return tokenized_question, tokenized_answer


# Load and preprocess the data
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
tokenized_articles_train,tokenized_summaries_train = load_and_preprocess_data("train-v2.0.json",num_prompts)
tokenized_articles_validation,tokenized_summaries_validation = load_and_preprocess_data("dev-v2.0.json", num_prompts)
device = "cpu"


# # Model Initialization
model = GPT2WithSoftPrompt(MODEL_NAME, num_prompts).to(device)


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 3b798cf7-0a44-4faa-b2c5-42b34752a753)')' thrown while requesting HEAD https://huggingface.co/gpt2/resolve/main/tokenizer_config.json


In [10]:
# test
model.eval()
total_val_loss = 0
with torch.no_grad():
    val_percentage_matched = 0
    val_percentage_matched_ct = 0
    for article, summary in tqdm(zip(tokenized_articles_validation, tokenized_summaries_validation), total=len(tokenized_articles_validation), desc="Validation", unit="batch"):
        input_ids = torch.tensor(article).to(device)
        labels = torch.tensor(summary).to(device)
        outputs = model(input_ids, prompt_id)

        ignore_index = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else -100
        val_loss = CrossEntropyLoss(ignore_index=ignore_index)(outputs.logits, labels)
        total_val_loss += val_loss.item()

        # Metrics
        set1 = set(torch.argmax(outputs.logits, dim=1).cpu().numpy())
        set2 = set(labels.cpu().numpy())

        # Calculate the intersection of sets
        intersection = set1.intersection(set2)

        # Calculate the percentage of indices in the first tensor that are also in the second tensor
        percentage = (len(intersection) / len(set1)) * 100
        val_percentage_matched += percentage
        val_percentage_matched_ct += 1


print("Val : % Exact Match: ",val_percentage_matched/val_percentage_matched_ct)
avg_val_loss = total_val_loss / len(tokenized_summaries_validation)
print("Val Loss : ",avg_val_loss)

Validation: 100%|██████████| 50/50 [00:24<00:00,  2.06batch/s]

Val : % Exact Match:  0.8412698412698413
Val Loss :  11.775735130310059


: 